In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=5406b32b8d39576ce4543862050c7640cfca4eb254b5482469d79dac7d8ad4fe
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pyspark.sql.functions
from pyspark.sql.functions import *
from pyspark.sql.functions import to_date, col, sum
from pyspark.sql.functions import to_date, dayofweek
from pyspark.sql.functions import mean, stddev
from pyspark.sql.functions import round
from pyspark.sql.functions import log
from pyspark.ml.feature import Binarizer
from pyspark.ml.feature import Bucketizer
from pyspark.ml.feature import VectorAssembler, Normalizer
from pyspark.sql.types import StringType, DateType
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [5]:
spark = SparkSession.builder.getOrCreate()

# Check Data

In [6]:
# Load dataset
df = spark.read.csv('/content/drive/MyDrive/BigData/final_exam/home-credit-default-risk/application_train.csv', header = True, inferSchema = True)

# First 5 rows of the dataframe
df.show(5)

+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+----------------+--------------------+--------------------+-----------------+--------------------------+----------+-------------+-----------------+---------------+-----------+----------+--------------+---------------+----------------+----------+----------+---------------+---------------+--------------------+---------------------------+--------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+--------------------+-------------------+------------------+-------------------+--------------+----------------+---------------------------+------------------+--------------+-------------+-------------+-------------+-------------+------------+--------------------+--------------+-----------------

In [7]:
df.count(), len(df.columns)

(307511, 122)

- Tệp dữ liệu có: 307511 dòng và 122 cột.

In [8]:
# check schema
df.printSchema()

root
 |-- SK_ID_CURR: integer (nullable = true)
 |-- TARGET: integer (nullable = true)
 |-- NAME_CONTRACT_TYPE: string (nullable = true)
 |-- CODE_GENDER: string (nullable = true)
 |-- FLAG_OWN_CAR: string (nullable = true)
 |-- FLAG_OWN_REALTY: string (nullable = true)
 |-- CNT_CHILDREN: integer (nullable = true)
 |-- AMT_INCOME_TOTAL: double (nullable = true)
 |-- AMT_CREDIT: double (nullable = true)
 |-- AMT_ANNUITY: double (nullable = true)
 |-- AMT_GOODS_PRICE: double (nullable = true)
 |-- NAME_TYPE_SUITE: string (nullable = true)
 |-- NAME_INCOME_TYPE: string (nullable = true)
 |-- NAME_EDUCATION_TYPE: string (nullable = true)
 |-- NAME_FAMILY_STATUS: string (nullable = true)
 |-- NAME_HOUSING_TYPE: string (nullable = true)
 |-- REGION_POPULATION_RELATIVE: double (nullable = true)
 |-- DAYS_BIRTH: integer (nullable = true)
 |-- DAYS_EMPLOYED: integer (nullable = true)
 |-- DAYS_REGISTRATION: double (nullable = true)
 |-- DAYS_ID_PUBLISH: integer (nullable = true)
 |-- OWN_CAR_AG

- Có 3 kiểu dữ liệu trong tệp tữ liệu là: `double`, `integer`, `string`

In [9]:
# kiểm tra ở cột target thì có nhiều
df.groupby("TARGET").count().show()

+------+------+
|TARGET| count|
+------+------+
|     1| 24825|
|     0|282686|
+------+------+



- 1: đang nợ tín dụng
- 0: không nợ tín dụng

- Số khách hàng nợ tín dụng là 24,825 chiếm 8% của cả tệp dữ liệu.
- Số khách hàng không nợ tín dụng là 282,686 chiếm 92% của cả tệp dữ liệu.

Khách hàng đang nợ tín dụng ít hơn rất nhiều so với khách hàng không nợ tín dụng, điều này có thể ảnh hưởng đến mô hình và và kết quả đánh giá vì nếu mô hình đoán sai 8% kia thì mô hình có độ chính xác là 92% nhưng đó không phải là mục đích của bài toán. -> Không sử dụng Accuracy để đánh giá mô hình

In [10]:
# kiểm tra có bao nhiêu phân tử null ở mỗi cột

# Tạo một list chứa số lượng giá trị null trong mỗi cột
null_counts = [sum(col(column).isNull().cast("int")).alias(column + "_null_count") for column in df.columns]

# Tạo DataFrame mới với số lượng giá trị null của mỗi cột
null_counts_df = df.agg(*null_counts)

null_counts_df.show()

+---------------------+-----------------+-----------------------------+----------------------+-----------------------+--------------------------+-----------------------+---------------------------+---------------------+----------------------+--------------------------+--------------------------+---------------------------+------------------------------+-----------------------------+----------------------------+-------------------------------------+---------------------+------------------------+----------------------------+--------------------------+----------------------+---------------------+-------------------------+--------------------------+---------------------------+---------------------+---------------------+--------------------------+--------------------------+-------------------------------+--------------------------------------+-------------------------------------+----------------------------------+-------------------------------------+-------------------------------------+-

# EDA

# Clean Data

In [11]:
df.show(5)

+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+----------------+--------------------+--------------------+-----------------+--------------------------+----------+-------------+-----------------+---------------+-----------+----------+--------------+---------------+----------------+----------+----------+---------------+---------------+--------------------+---------------------------+--------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+--------------------+-------------------+------------------+-------------------+--------------+----------------+---------------------------+------------------+--------------+-------------+-------------+-------------+-------------+------------+--------------------+--------------+-----------------

## Drop columns with more than 35% missing

In [12]:
def column_dropper(df, threshold):
    # Takes a dataframe and threshold for missing values. Returns a dataframe.
    total_records = df.count()
    for col in df.columns:
        # Calculate the percentage of missing values
        missing = df.where(df[col].isNull()).count()
        missing_percent = missing / total_records
        # Drop column if percent of missing is more than threshold
        if missing_percent > threshold:
            df = df.drop(col)
    return df

df = column_dropper(df, 0.35)

In [13]:
df.count(), len(df.columns)

(307511, 73)

Sau khi xóa các cột có

In [15]:
# kiểm tra có bao nhiêu phân tử null ở mỗi cột

# Tạo một list chứa số lượng giá trị null trong mỗi cột
null_counts = [sum(col(column).isNull().cast("int")).alias(column + "_null_count") for column in df.columns]

# Tạo DataFrame mới với số lượng giá trị null của mỗi cột
null_counts_df = df.agg(*null_counts)

null_counts_df.show()

+---------------------+-----------------+-----------------------------+----------------------+-----------------------+--------------------------+-----------------------+---------------------------+---------------------+----------------------+--------------------------+--------------------------+---------------------------+------------------------------+-----------------------------+----------------------------+-------------------------------------+---------------------+------------------------+----------------------------+--------------------------+---------------------+-------------------------+--------------------------+---------------------------+---------------------+---------------------+--------------------------+--------------------------+-------------------------------+--------------------------------------+-------------------------------------+----------------------------------+-------------------------------------+-------------------------------------+------------------------

In [16]:
# drop cột WEEKDAY_APPR_PROCESS_START: cột không cần thiết
df = df.drop("WEEKDAY_APPR_PROCESS_START")

## Fill Columns which have null:

In [17]:
df.filter(df["AMT_ANNUITY"].isNull()).show()

+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+--------------------+--------------------+--------------------+-------------------+--------------------------+----------+-------------+-----------------+---------------+----------+--------------+---------------+----------------+----------+----------+--------------------+---------------+--------------------+---------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+--------------------+-------------------+-------------------+------------------------+------------------------+------------------------+------------------------+----------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+------------

### AMT_ANNUITY, AMT_GOOD_PRICE

Lấy cột AMT_ANNUITY chia cho AMT_GOOD_PRICE rồi lấy tỷ lệ chung bình. rồi lấy tỷ lệ đấy nhân với AMT_GOOD_PRICE để điền giá trị thiếu của cột AMT_ANNUITY.

In [18]:
rate_mean = df.select((col("AMT_ANNUITY") / col("AMT_GOODS_PRICE")).alias("Ratio")) \
           .agg({"Ratio": "sum"}) \
           .collect()[0][0] / df.count()
print(rate_mean)

0.05986277738286673


In [19]:
df = df.withColumn("AMT_ANNUITY", when(col("AMT_ANNUITY").isNull(),
                                          col("AMT_GOODS_PRICE") * rate_mean).otherwise(col("AMT_ANNUITY")))

In [20]:
df.filter(col("AMT_ANNUITY").isNull()).count()

0

In [21]:
# fill cột AMT_GOODS_PRICE với rate_mean: AMT_ANNUITY / rate_mean
df = df.withColumn("AMT_GOODS_PRICE", when(col("AMT_GOODS_PRICE").isNull(),
                                          col("AMT_ANNUITY") / rate_mean).otherwise(col("AMT_GOODS_PRICE")))

In [22]:
df.filter(col("AMT_GOODS_PRICE").isNull()).count()

0

### NAME_TYPE_SUITE

In [23]:
df.groupby('NAME_TYPE_SUITE').count().show()

+---------------+------+
|NAME_TYPE_SUITE| count|
+---------------+------+
|Spouse, partner| 11370|
|        Other_A|   866|
|           NULL|  1292|
|         Family| 40149|
|Group of people|   271|
|        Other_B|  1770|
|  Unaccompanied|248526|
|       Children|  3267|
+---------------+------+



In [24]:
# điền các giá trị thiếu là các mode
mode_value = df.groupBy("NAME_TYPE_SUITE").count().sort(col("count").desc()).first()[0]
mode_value

'Unaccompanied'

In [25]:
df = df.fillna(mode_value, subset=["NAME_TYPE_SUITE"])

In [26]:
df.filter(col("NAME_TYPE_SUITE").isNull()).count()

0

### OCCUPATION_TYPE

In [27]:

df.groupby('OCCUPATION_TYPE').count().show()

+--------------------+-----+
|     OCCUPATION_TYPE|count|
+--------------------+-----+
|                NULL|96391|
|            Managers|21371|
|            HR staff|  563|
|      Medicine staff| 8537|
|         Accountants| 9813|
|            Laborers|55186|
|      Cleaning staff| 4653|
|Private service s...| 2652|
|             Drivers|18603|
|         Sales staff|32102|
|       Realty agents|  751|
|            IT staff|  526|
|      Security staff| 6721|
|         Secretaries| 1305|
|  Low-skill Laborers| 2093|
|          Core staff|27570|
|       Cooking staff| 5946|
|High skill tech s...|11380|
|Waiters/barmen staff| 1348|
+--------------------+-----+



Giá trị thiếu là giá trị xuất hiện nhiều nhất, ta sẽ điền là Other, vì sau này sẽ có nhiều ngành nghề khác mà không có dữ liệu trong tập này, thì sẽ điền Other.

In [28]:
df = df.fillna('Other', subset=["OCCUPATION_TYPE"])

In [29]:
df.filter(col("OCCUPATION_TYPE").isNull()).count()

0

### CNT_FAM_MEMBERS

In [30]:
df.filter(df["CNT_FAM_MEMBERS"].isNull()).show()

+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+-----------------+---------------+--------------------+-------------------+------------------+-------------------+--------------------------+----------+-------------+-----------------+---------------+----------+--------------+---------------+----------------+----------+----------+---------------+---------------+--------------------+---------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+--------------------+------------------+------------------+------------------------+------------------------+------------------------+------------------------+----------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+----------------+---

In [31]:
df.groupby('CNT_FAM_MEMBERS').count().show()

+---------------+------+
|CNT_FAM_MEMBERS| count|
+---------------+------+
|            8.0|    20|
|            7.0|    81|
|           NULL|     2|
|            1.0| 67847|
|            4.0| 24697|
|           14.0|     2|
|            3.0| 52601|
|            2.0|158357|
|           10.0|     3|
|           13.0|     1|
|            6.0|   408|
|           20.0|     2|
|            5.0|  3478|
|            9.0|     6|
|           12.0|     2|
|           11.0|     1|
|           15.0|     1|
|           16.0|     2|
+---------------+------+



In [32]:
# Lấy median để điền vào null
median_value = df.approxQuantile("CNT_FAM_MEMBERS", [0.5], 0.25)[0]

df = df.fillna(median_value, subset=["CNT_FAM_MEMBERS"])

In [33]:
df.filter(col("CNT_FAM_MEMBERS").isNull()).count()

0

### EXT_SOURCE_2, EXT_SOURCE_3

In [34]:
# Điền giá trị trung vào 2 cột.

# EXT_SOURCE_2
# Tính giá trị trung bình của cột "EXT_SOURCE_2"
mean_value = df.select(mean("EXT_SOURCE_2")).collect()[0][0]

# Điền giá trị null trong cột "EXT_SOURCE_2" bằng giá trị trung bình
df = df.fillna(mean_value, subset=["EXT_SOURCE_2"])

In [35]:
# EXT_SOURCE_3
# Tính giá trị trung bình của cột "EXT_SOURCE_3"
mean_value = df.select(mean("EXT_SOURCE_3")).collect()[0][0]

# Điền giá trị null trong cột "EXT_SOURCE_3" bằng giá trị trung bình
df = df.fillna(mean_value, subset=["EXT_SOURCE_3"])

### OBS_30_CNT_SOCIAL_CIRCLE, DEF_30_CNT_SOCIAL_CIRCLE, OBS_60_CNT_SOCIAL_CIRCLE, DEF_60_CNT_SOCIAL_CIRCLE

In [36]:
# Điền tất cả các giá trị null = 0
df = df.fillna(0, subset=["OBS_30_CNT_SOCIAL_CIRCLE", "DEF_30_CNT_SOCIAL_CIRCLE",
                          "OBS_60_CNT_SOCIAL_CIRCLE", "DEF_60_CNT_SOCIAL_CIRCLE"])

### DAYS_LAST_PHONE_CHANGE

In [37]:
# điền giá trị trung bình
# Tính giá trị trung bình của cột "DAYS_LAST_PHONE_CHANGE"
mean_value = df.select(mean("DAYS_LAST_PHONE_CHANGE")).collect()[0][0]

# Điền giá trị null trong cột "DAYS_LAST_PHONE_CHANGE" bằng giá trị trung bình
df = df.fillna(mean_value, subset=["DAYS_LAST_PHONE_CHANGE"])

### AMT_REQ_CREDIT_BUREAU_HOUR, AMT_REQ_CREDIT_BUREAU_DAY, AMT_REQ_CREDIT_BUREAU_WEEK, AMT_REQ_CREDIT_BUREAU_MON, AMT_REQ_CREDIT_BUREAU_QRT, AMT_REQ_CREDIT_BUREAU_YEAR

In [ ]:
df.groupby('AMT_REQ_CREDIT_BUREAU_YEAR').count().show()

In [ ]:
df.filter(df["AMT_REQ_CREDIT_BUREAU_YEAR"].isNull()).show()

+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+--------------------+--------------------+--------------------+-----------------+--------------------------+----------+-------------+-----------------+---------------+----------+--------------+---------------+----------------+----------+----------+--------------------+---------------+--------------------+---------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+--------------------+-------------------+------------------+------------------------+------------------------+------------------------+------------------------+----------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------

In [ ]:
# DỮ lại cột AMT_REQ_CREDIT_BUREAU_QRT
df = df.drop("AMT_REQ_CREDIT_BUREAU_HOUR", "AMT_REQ_CREDIT_BUREAU_DAY",
             "AMT_REQ_CREDIT_BUREAU_WEEK", "AMT_REQ_CREDIT_BUREAU_MON",
             "AMT_REQ_CREDIT_BUREAU_YEAR")

In [ ]:
df.groupby('AMT_REQ_CREDIT_BUREAU_QRT').count().show()

+-------------------------+------+
|AMT_REQ_CREDIT_BUREAU_QRT| count|
+-------------------------+------+
|                      8.0|     7|
|                      0.0|215417|
|                      7.0|     7|
|                     NULL| 41519|
|                      1.0| 33862|
|                      4.0|   476|
|                      3.0|  1717|
|                      2.0| 14412|
|                      6.0|    28|
|                      5.0|    64|
|                     19.0|     1|
|                    261.0|     1|
+-------------------------+------+



In [ ]:
# Điền các giá trị null  = 0
df = df.fillna(0, subset=["AMT_REQ_CREDIT_BUREAU_QRT"])

In [ ]:
# kiểm tra có bao nhiêu phân tử null ở mỗi cột

# Tạo một list chứa số lượng giá trị null trong mỗi cột
null_counts = [sum(col(column).isNull().cast("int")).alias(column + "_null_count") for column in df.columns]

# Tạo DataFrame mới với số lượng giá trị null của mỗi cột
null_counts_df = df.agg(*null_counts)

null_counts_df.show()

+---------------------+-----------------+-----------------------------+----------------------+-----------------------+--------------------------+-----------------------+---------------------------+---------------------+----------------------+--------------------------+--------------------------+---------------------------+------------------------------+-----------------------------+----------------------------+-------------------------------------+---------------------+------------------------+----------------------------+--------------------------+---------------------+-------------------------+--------------------------+---------------------------+---------------------+---------------------+--------------------------+--------------------------+-------------------------------+--------------------------------------+----------------------------------+-------------------------------------+-------------------------------------+--------------------------------------+-----------------------

Dữ liệu đã điền đầy đủ null

## Kiểm tra Logic của các cột kiểu dữ liệu số và sửa lỗi dữ liệu
---



In [ ]:
df.show()

+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+--------------------+--------------------+--------------------+-----------------+--------------------------+----------+-------------+-----------------+---------------+----------+--------------+---------------+----------------+----------+----------+---------------+---------------+--------------------+---------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+--------------------+-------------------+-------------------+------------------------+------------------------+------------------------+------------------------+----------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+----------------+--

In [ ]:
df.groupby("CODE_GENDER").count().show()

+-----------+------+
|CODE_GENDER| count|
+-----------+------+
|          F|202448|
|          M|105059|
|        XNA|     4|
+-----------+------+



In [ ]:
# bỏ các dòng có CODE_GENDER = "XNA"
df = df.filter(df["CODE_GENDER"] != "XNA")
df.groupby("CODE_GENDER").count().show()

+-----------+------+
|CODE_GENDER| count|
+-----------+------+
|          F|202448|
|          M|105059|
+-----------+------+



### AMT_INCOME_TOTAL, AMT_CREDIT, AMT_ANNUITY, AMT_GOODS_PRICE

In [ ]:
columns_to_describe = ["AMT_INCOME_TOTAL", "AMT_CREDIT",
                       "AMT_ANNUITY", "AMT_GOODS_PRICE"]
df.select(columns_to_describe).describe().show()

+-------+------------------+-----------------+------------------+-----------------+
|summary|  AMT_INCOME_TOTAL|       AMT_CREDIT|       AMT_ANNUITY|  AMT_GOODS_PRICE|
+-------+------------------+-----------------+------------------+-----------------+
|  count|            307507|           307507|            307507|           307507|
|   mean|168797.68577929938|599028.5967327573|27108.798547456114|538122.1378510538|
| stddev| 237124.6273197452|402492.6018586199|14494.011039141047| 369424.508025762|
|    min|           25650.0|          45000.0|            1615.5|          40500.0|
|    max|            1.17E8|        4050000.0|          258025.5|        4050000.0|
+-------+------------------+-----------------+------------------+-----------------+



In [ ]:
df.count(), len(df.columns)

(307507, 67)

Loại các giá trị có số tiền vay vượt hạn mức cho phép.

In [ ]:
df = df.filter(df["AMT_CREDIT"] >= df["AMT_GOODS_PRICE"])

In [ ]:
df.count(), len(df.columns)

(307248, 67)

In [ ]:
df.groupby("TARGET").count().show()

+------+------+
|TARGET| count|
+------+------+
|     1| 24812|
|     0|282436|
+------+------+



In [ ]:
df = df.filter(df["AMT_GOODS_PRICE"] >= df["AMT_ANNUITY"])

In [ ]:
df.count(), len(df.columns)

(307248, 67)

Biến đổi các giá trị âm thành dương

In [ ]:
df.show()

+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+--------------------+--------------------+--------------------+-----------------+--------------------------+----------+-------------+-----------------+---------------+----------+--------------+---------------+----------------+----------+----------+---------------+---------------+--------------------+---------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+--------------------+-------------------+-------------------+------------------------+------------------------+------------------------+------------------------+----------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+----------------+--

In [ ]:
negative_value_columns = ["DAYS_BIRTH", "DAYS_EMPLOYED",
                          "DAYS_REGISTRATION", "DAYS_ID_PUBLISH", "DAYS_LAST_PHONE_CHANGE"]

for column in negative_value_columns:
    df = df.withColumn(column, when(col(column) < 0, -col(column)).otherwise(col(column)))

In [ ]:
df.select(negative_value_columns).show(5)

+----------+-------------+-----------------+---------------+----------------------+
|DAYS_BIRTH|DAYS_EMPLOYED|DAYS_REGISTRATION|DAYS_ID_PUBLISH|DAYS_LAST_PHONE_CHANGE|
+----------+-------------+-----------------+---------------+----------------------+
|      9461|          637|           3648.0|           2120|                1134.0|
|     16765|         1188|           1186.0|            291|                 828.0|
|     19046|          225|           4260.0|           2531|                 815.0|
|     19005|         3039|           9833.0|           2437|                 617.0|
|     19932|         3038|           4311.0|           3458|                1106.0|
+----------+-------------+-----------------+---------------+----------------------+
only showing top 5 rows



In [ ]:
df.count(), len(df.columns)

(307248, 67)

Loại các giá trị ở cột DAYS_EMPLOYED, DAYS_REGISTRATION, DAYS_ID_PUBLISH nếu có giá trị lớn hơn ở cột DAYS_BIRTH

In [ ]:

columns = ["DAYS_EMPLOYED", "DAYS_REGISTRATION", "DAYS_ID_PUBLISH"]

for column in columns:
    df = df.filter(df["DAYS_BIRTH"] >= df[column])
    print(df.count(), len(df.columns))

251908 67
251907 67
251907 67


In [ ]:
df.show(5)

+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+----------------+--------------------+--------------------+-----------------+--------------------------+----------+-------------+-----------------+---------------+----------+--------------+---------------+----------------+----------+----------+---------------+---------------+--------------------+---------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+--------------------+------------------+-------------------+------------------------+------------------------+------------------------+------------------------+----------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+----------------+-------

### Lọc dữ liệu Ngoại Lai

In [ ]:
print(df.count(), len(df.columns))

251907 67


In [ ]:
# Calculate values used for outlier filtering
mean_val = df.agg({'AMT_GOODS_PRICE': 'mean'}).collect()[0][0]
stddev_val = df.agg({'AMT_GOODS_PRICE': 'stddev'}).collect()[0][0]

# Create three standard deviation (μ ± 3σ) lower and upper bounds for data
low_bound = mean_val - (3 * stddev_val)
hi_bound = mean_val + (3 * stddev_val)

# Filter the data to fit between the lower and upper bounds
# lọc dữ liệu ngoại lại

df_not_out = df.where((df['AMT_GOODS_PRICE'] < hi_bound) & (df['AMT_GOODS_PRICE'] > low_bound))
df_is_out = df.where((df['AMT_GOODS_PRICE'] >= hi_bound) | (df['AMT_GOODS_PRICE'] <= low_bound))

In [ ]:
print(df_not_out.count(), len(df_not_out.columns))
print(df_is_out.count(), len(df_is_out.columns))

248324 67
3583 67


In [ ]:
print(df.count(), len(df.columns))

251907 67


Dữ liệu đã sạch ở mức cơ bản

# Điều chỉnh Dữ liệu

In [ ]:
# Lưu dữ liệu vào file mới
df_not_out.write.csv("/content/data_clean.csv", header = True, mode = "overwrite")

In [ ]:
# Load dataset
df_clean = spark.read.csv('/content/data_clean.csv', header = True, inferSchema = True)



In [ ]:
df_clean.printSchema()

root
 |-- SK_ID_CURR: integer (nullable = true)
 |-- TARGET: integer (nullable = true)
 |-- NAME_CONTRACT_TYPE: string (nullable = true)
 |-- CODE_GENDER: string (nullable = true)
 |-- FLAG_OWN_CAR: string (nullable = true)
 |-- FLAG_OWN_REALTY: string (nullable = true)
 |-- CNT_CHILDREN: integer (nullable = true)
 |-- AMT_INCOME_TOTAL: double (nullable = true)
 |-- AMT_CREDIT: double (nullable = true)
 |-- AMT_ANNUITY: double (nullable = true)
 |-- AMT_GOODS_PRICE: double (nullable = true)
 |-- NAME_TYPE_SUITE: string (nullable = true)
 |-- NAME_INCOME_TYPE: string (nullable = true)
 |-- NAME_EDUCATION_TYPE: string (nullable = true)
 |-- NAME_FAMILY_STATUS: string (nullable = true)
 |-- NAME_HOUSING_TYPE: string (nullable = true)
 |-- REGION_POPULATION_RELATIVE: double (nullable = true)
 |-- DAYS_BIRTH: integer (nullable = true)
 |-- DAYS_EMPLOYED: integer (nullable = true)
 |-- DAYS_REGISTRATION: double (nullable = true)
 |-- DAYS_ID_PUBLISH: integer (nullable = true)
 |-- FLAG_MOBIL

In [ ]:
# First 5 rows of the dataframe
df_clean.show(5)

+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+----------------+--------------------+--------------------+-----------------+--------------------------+----------+-------------+-----------------+---------------+----------+--------------+---------------+----------------+----------+----------+---------------+---------------+--------------------+---------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+--------------------+------------------+-------------------+------------------------+------------------------+------------------------+------------------------+----------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+----------------+-------

In [ ]:
df_clean.describe().show()

In [ ]:
df_clean.groupby("NAME_TYPE_SUITE").count().show()

+---------------+------+
|NAME_TYPE_SUITE| count|
+---------------+------+
|              1| 45983|
|              0|205924|
+---------------+------+



ta sẽ đổi các giá trị cột thành:
  - Những người không có người đi cùng sẽ là: `Unaccompanied` -> 0
  - Những người có người đi cùng sẽ là: `Accompanied` -> 1

In [ ]:

df_clean = df_clean.withColumn("NAME_TYPE_SUITE", when(df_clean["HAS_ACCOMPANIED"] == "Unaccompanied", 0).otherwise(1))


In [ ]:
# biển đổi dữ liệu (Y/N) -> (1/0)
df_clean = df_clean.withColumn("FLAG_OWN_CAR", when(df_clean["FLAG_OWN_CAR"] == "Y", 1).otherwise(0))

df_clean = df_clean.withColumn("FLAG_OWN_REALTY", when(df_clean["FLAG_OWN_REALTY"] == "Y", 1).otherwise(0))

# biến đổi cột CODE_GENDER: M->1; F->0
df_clean = df_clean.withColumn("CODE_GENDER", when(df_clean["CODE_GENDER"] == "M", 1).otherwise(0))



In [ ]:
df_clean.groupby("NAME_CONTRACT_TYPE").count().show()

+------------------+------+
|NAME_CONTRACT_TYPE| count|
+------------------+------+
|   Revolving loans| 25684|
|        Cash loans|226223|
+------------------+------+



In [ ]:
# biến đổi dữ liệu cột NAME_CONTRACT_TYPE:
# Cash loans -> 1; Revolving loans -> 0
df_clean = df_clean.withColumn("NAME_CONTRACT_TYPE", when(df_clean["NAME_CONTRACT_TYPE"] == "Cash loans", 1).otherwise(0))


In [ ]:
df_clean.show(5)

+----------+------+------------------+-----------+------------+---------------+------------+---------------+----------------+--------------------+--------------------+-----------------+--------------------------+----------+--------------+---------------+----------------+----------+----------+---------------+---------------+--------------------+---------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+--------------------+------------------+-------------------+------------------------+------------------------+------------------------+------------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+---------

## Scaler

In [ ]:
df_clean.show(5)

+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+----------------+-------------------+------------------+-----------------+--------------------------+----------+-------------+-----------------+---------------+----------+--------------+---------------+----------------+----------+----------+---------------+---------------+--------------------+---------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+--------------------+------------------+-------------------+------------------------+------------------------+------------------------+------------------------+----------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+----------------+----------

In [ ]:
df_clean.describe().show()

In [ ]:
def min_max_scaler(df, cols_to_scale):
    # Takes a dataframe and list of columns to minmax scale. Returns a dataframe.
    for col in cols_to_scale:
        # Define min and max values and collect them
        max_days = df.agg({col: 'max'}).collect()[0][0]
        min_days = df.agg({col: 'min'}).collect()[0][0]
        new_column_name = 'scaled_' + col
        # Create a new column based off the scaled data
        df = df.withColumn(new_column_name, (df[col] - min_days) / (max_days - min_days))

    return df

In [ ]:
def log10_scaler(df, cols_to_scale):
    for col in cols_to_scale:
        df = df.withColumn("Log10_" + col, log10(df[col]))
    return df

In [ ]:
columns_minmax= ["AMT_INCOME_TOTAL", "AMT_CREDIT",
                 "AMT_ANNUITY", "AMT_GOODS_PRICE"]

columns_log = ["DAYS_BIRTH", "DAYS_EMPLOYED", "DAYS_REGISTRATION",
               "DAYS_ID_PUBLISH", "DAYS_LAST_PHONE_CHANGE"]

In [ ]:
# biển đổi các cột dữ liệu
df_clean = min_max_scaler(df_clean, columns_minmax)
df_clean = log10_scaler(df_clean, columns_log)

In [ ]:
# Xóa các cột đã được chuẩn hóa
drop_columns = columns_minmax + columns_log
df_clean = df_clean.drop(*drop_columns)

In [ ]:
df_clean.show(5)

+----------+------+------------------+-----------+------------+---------------+------------+---------------+----------------+--------------------+--------------------+-----------------+--------------------------+----------+--------------+---------------+----------------+----------+----------+---------------+---------------+--------------------+---------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+--------------------+------------------+-------------------+------------------------+------------------------+------------------------+------------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+---------

In [ ]:
df_clean.show()

+----------+------+------------------+-----------+------------+---------------+------------+---------------+--------------------+--------------------+--------------------+-----------------+--------------------------+----------+--------------+---------------+----------------+----------+----------+---------------+---------------+--------------------+---------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+--------------------+-------------------+-------------------+------------------------+------------------------+------------------------+------------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----

# Feature Engineer

## One Hot En

In [ ]:
df_clean.show()

+----------+------+------------------+-----------+------------+---------------+------------+---------------+--------------------+--------------------+--------------------+-----------------+--------------------------+----------+--------------+---------------+----------------+----------+----------+---------------+---------------+--------------------+---------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+--------------------+-------------------+-------------------+------------------------+------------------------+------------------------+------------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----

In [ ]:
len

In [ ]:
# Đổi các giá trị chuỗi thành số

# tên cột chuỗi có dạng dữ liệu string
string_columns = [col_name for col_name, col_type in df_clean.dtypes if col_type == "string"]

In [ ]:
string_columns

['NAME_CONTRACT_TYPE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'ORGANIZATION_TYPE']

In [ ]:
# chuyển list thành dataframe mới
def move(df, column, unique_values_dict):
    # chuyển dữ liệu thành list
    col_list = df.select(column).rdd.flatMap(lambda x: x).collect()

    # ánh xạ các giá trị là số
    col_num = list(map(lambda x:unique_values_dict[x], col_list ))

    # chuyển dữ liệu thành tuple con để tạo dataframe
    data = [(value,) for value in col_num]
    # spark = SparkSession.builder.appName("UpdateColumnValues").getOrCreate()
    df_new = spark.createDataFrame(data, [column + "_ORDINAL"])

    # tạo cột giả để nói 2 cột
    df_new = df_new.withColumn("ID", monotonically_increasing_id())
    df = df.withColumn("ID", monotonically_increasing_id())

    joined_df = df.join(df_new, on = "ID", how = "inner")
    joined_df = joined_df.drop("ID")
    spark.stop()
    return joined_df


# Đổi các giá trị chuỗi thành số
def ordinal(df, columns):
    for column in columns:
        unique_values_list = df.select(column).distinct().rdd.flatMap(lambda x: x).collect()
        unique_values_dict = {}
        for i, value in enumerate(unique_values_list, start=1):
            unique_values_dict[value] = i
        df = move(df, column, unique_values_dict)
        # xóa cột gốc
        df =df.drop(column)

    return df

In [ ]:
df_clean = ordinal(df_clean, string_columns)

Py4JJavaError: An error occurred while calling o10659.javaToPython.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServerConn

In [ ]:

colum = "NAME_CONTRACT_TYPE"
unique_values_list = df.select(column).distinct().rdd.flatMap(lambda x: x).collect()
unique_values_dict = {}
for i, value in enumerate(unique_values_list, start=1):
    unique_values_dict[value] = i
# df = move(df, column, unique_values_dict, "SK_ID_CURR")
# # xóa cột gốc
# df =df.drop(column)



In [ ]:

# # chuyển dữ liệu thành list
# col_list = df.select(column).rdd.flatMap(lambda x: x).collect()
# # col_list

# # # ánh xạ các giá trị là số
# col_num = list(map(lambda x:unique_values_dict[x], col_list ))
# col_num

# # chuyển dữ liệu thành tuple con để tạo dataframe
# data = [(value,) for value in col_num]
# spark = SparkSession.builder.appName("UpdateColumnValues").getOrCreate()
# df_new = spark.createDataFrame(data, [column + "_ordinal"])
# df_new.show()

# # tạo cột giả để nói 2 cột
df_new = df_new.withColumn("ID", monotonically_increasing_id() )
df_1
# joined_df = df.join(df_new, on = "SK_ID_CURR", how = "inner")
# # spark.stop()

AnalysisException: [MISSING_ATTRIBUTES.RESOLVED_ATTRIBUTE_MISSING_FROM_INPUT] Resolved attribute(s) "SK_ID_CURR" missing from "NAME_INCOME_TYPE_ordinal" in operator !Project [NAME_INCOME_TYPE_ordinal#46021L, SK_ID_CURR#17 AS SK_ID_CURR#46023]. ;
!Project [NAME_INCOME_TYPE_ordinal#46021L, SK_ID_CURR#17 AS SK_ID_CURR#46023]
+- LogicalRDD [NAME_INCOME_TYPE_ordinal#46021L], false


In [ ]:
# def ordinal(df, columns):
#     for column in columns:
#         # Tạo DataFrame tạm thời chứa các giá trị đã chuyển đổi
#         ordinal_df = df.select(column).distinct().withColumn(column + "_ORDINAL", monotonically_increasing_id())

#         # Kết hợp DataFrame tạm thời với DataFrame gốc thông qua cột ID
#         df = df.join(ordinal_df, on=column, how="left")

#         # Xóa cột gốc
#         df = df.drop(column)

#     return df

spark = SparkSession.builder.appName("OrdinalConversion").getOrCreate()
df = ordinal(df, string_columns )

In [ ]:
df.show()

Py4JJavaError: An error occurred while calling o10373.showString.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServerConnec

In [ ]:
df.select("SK_ID_CURR").show()

+----------+
|SK_ID_CURR|
+----------+
|    100002|
|    100003|
|    100004|
|    100006|
|    100007|
|    100008|
|    100009|
|    100010|
|    100012|
|    100014|
|    100016|
|    100017|
|    100018|
|    100019|
|    100020|
|    100021|
|    100022|
|    100023|
|    100024|
|    100025|
+----------+
only showing top 20 rows



In [ ]:
df_new = df_new.crossJoin(df.select("SK_ID_CURR").limit(1))

In [ ]:
df.select("SK_ID_CURR").limit(1).show()

+----------+
|SK_ID_CURR|
+----------+
|    100002|
+----------+



In [ ]:
df_new.withColumn("ID",monotonically_increasing_id()).show()
df.withColumn("ID", monotonically_increasing_id()).show()

+------------------------+---+
|NAME_INCOME_TYPE_ordinal| ID|
+------------------------+---+
|                       6|  0|
|                       5|  1|
|                       6|  2|
|                       6|  3|
|                       6|  4|
|                       5|  5|
|                       2|  6|
|                       5|  7|
|                       6|  8|
|                       6|  9|
|                       6| 10|
|                       6| 11|
|                       6| 12|
|                       6| 13|
|                       6| 14|
|                       6| 15|
|                       6| 16|
|                       5| 17|
|                       6| 18|
|                       2| 19|
+------------------------+---+
only showing top 20 rows

+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+--------------------+--------------------+--------------------+---

In [ ]:
df_new = df_new.withColumn("SK_ID_CURR", lit(df.select("SK_ID_CURR").collect()[0][0]))

In [ ]:
 lit(df.select("SK_ID_CURR").collect()[0][0])

Column<'100002'>